In [ ]:
import keras
import numpy as np
import tensorflow as tf
from keras.src.optimizers import Adam
from keras.src.layers import Dense
from keras.src.layers import Flatten
from keras.src.optimizers import RMSprop
from keras import Sequential, Input
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.src.layers import Dropout
from keras.src.layers import BatchNormalization
import pandas as pd
from utils.tensorflow_preprocessing import prepare_image_dataset
from keras.src.callbacks import ProgbarLogger
from keras.src.callbacks import EarlyStopping

In [ ]:
# Load dataset
train_df = pd.read_csv('../data/processed/csv/train_df.csv')
val_df = pd.read_csv('../data/processed/csv/val_df.csv')
test_df = pd.read_csv('../data/processed/csv/test_df.csv')

In [ ]:
train_dataset, label_encoder = prepare_image_dataset(train_df, img_height=256, img_width=256, batch_size=32)
val_dataset, _ = prepare_image_dataset(val_df, img_height=256, img_width=256, batch_size=32, label_encoder=label_encoder)
test_dataset, _ = prepare_image_dataset(test_df, img_height=256, img_width=256, batch_size=32, label_encoder=label_encoder)

In [ ]:
# Define input shape and number of classes
INPUT_SHAPE = (256, 256, 3)  # Example input shape (image dimensions and channels)
NUM_CLASSES = 6  # Number of output classes (furniture types)

In [ ]:
# Define base model
def create_model(num_layers=2, num_hidden_neurons=None, optimizer=Adam(0.001)):
    if num_hidden_neurons is None:
        num_hidden_neurons = [64, 32]
    model = Sequential()
    model.add(Input(shape=INPUT_SHAPE))
    model.add(Flatten()),
    for i in range(num_layers):
        model.add(Dense(num_hidden_neurons[i], activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(rate=0.2))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    model.compile(optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)
    return model, early_stopping

In [ ]:
model, early_stopping = create_model(num_layers=3, num_hidden_neurons=[128, 64, 32], optimizer=Adam(0.001))

In [ ]:
history = model.fit(train_dataset, epochs=50, batch_size=32, callbacks=[ProgbarLogger(), early_stopping])

In [ ]:
train_loss, train_acc = model.evaluate(train_dataset, verbose=2)
print(f"Test Accuracy: {train_acc * 100:.2f}%")
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)
print(f"Test Accuracy: {test_acc * 100:.2f}%")
val_loss, val_acc = model.evaluate(val_dataset, verbose=2)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")